# Iterated Prisoner Dilemma Problem

## Computer will roll a dice to decide how to make a move

## Computer Plays without any intelligence

1. Generate Random Numbers (I need more like a coin toss)
2. 2 players -> Generate Player 1 -> Shadow Strategy for Player 2 -> Player 2 Move
3. Assign scores to each game

## Result - Against a random opponent persist and take your stance

In [38]:
from mock import patch
import enum
import random
import abc

In [170]:
class Coin(enum.Flag):
    HEAD=True  # Assume this is co-operate
    TAIL=False # Assume this is defecting 

In [7]:
class MyDice(object):
    @classmethod
    def get_random(self):
        return random.randint(0,1)
    
    @classmethod
    def roll(self):
        value = self.get_random()
        return Coin.HEAD if value == 1 else Coin.TAIL

In [29]:
class TargetPlayer(object):
    def __init__(self, max_count):
        self.max_count = max_count
        self.dice = MyDice()
    
    def __iter__(self):
        self.count = 0
        return self
    
    def __next__(self):
        if self.count >= self.max_count:
            raise StopIteration
        self.count += 1
        return self.dice.roll()

In [49]:
class Strategy(abc.ABC):
    def __init__(self, initial, target_player_sim):
        self.initial = initial
        self.target_player_sim = target_player_sim
    
    @abc.abstractmethod
    def strategy(self):
        pass

In [116]:
class TitForTat(Strategy):
    def strategy(self):
        res = []
        # First move peace by me always
        res.append(self.initial)
        for prev_val in self.target_player_sim[:-1]: # Skip last he will play it any way
            res.append(prev_val)
            
        return res

In [122]:
class Punisher(Strategy):
    def strategy(self):
        res = []
        # First move peace by me always
        res.append(self.initial)
        flag = False
        for prev_val in self.target_player_sim[:-1]: # Skip last he will play it any way
            if prev_val == Coin.TAIL:
                flag = True
            if flag:
                res.append(Coin.TAIL)
            else:
                res.append(Coin.HEAD)
            
        return res

In [163]:
DEFECT_WIN_SUM = 250
DEFECT_LOSS_SUM = 100
COOPERATE_WIN_SUM = 150
COOPERATE_LOSS_SUM = -500

def get_score_util(me, him):
    if me and him:
        return (COOPERATE_WIN_SUM, COOPERATE_WIN_SUM)
    if not me and him:
        return (DEFECT_WIN_SUM, COOPERATE_LOSS_SUM)
    if me and not him:
        return (COOPERATE_LOSS_SUM, DEFECT_WIN_SUM)
    if not me and not him:
        return (DEFECT_LOSS_SUM, DEFECT_LOSS_SUM)
    raise ValueError('Check if all possible combinations are not covered')

In [164]:
import unittest

class TestMyDice(unittest.TestCase):
    """Funny test, just to tdd, no significance"""
    def test_roll(self):
        heads = 0
        tails = 0
        count = 500000
        dice = MyDice()
        for i in range(count):
            value = dice.roll()
            if value == Coin.HEAD:
                heads += 1
            else:
                tails += 1
        self.assertEqual(count, heads + tails)
                
    def test_get_random(self):
        dice = MyDice()
        value = dice.get_random()
        self.assertIn(value, [0,1])
        value = dice.roll()
        self.assertIn(value, Coin)
    
    @patch.object(MyDice, 'get_random')
    def test_integration(self, mock_get_random):  
        mock_get_random.return_value=1
        dice =  MyDice()
        value = dice.roll()
        self.assertEqual(value, Coin.HEAD)
        mock_get_random.return_value=0
        value = dice.roll()
        self.assertEqual(value, Coin.TAIL)

class TestCoin(unittest.TestCase):
    """Testcase for the enum Coin"""
    def test_truth(self):
        self.assertEqual(Coin.HEAD.value, True)
        self.assertEqual(Coin.TAIL.value, False)

class TestTargetPlayer(unittest.TestCase):
    """Testcase for TargetPlater class"""
    def test_play(self):
        count = 10
        for value in TargetPlayer(count):
            self.assertIn(value, Coin)
    
    def test_max_count(self):
        count = 0
        target_count = 15
        for x in TargetPlayer(target_count):
            count += 1
        self.assertEqual(count, target_count)

class TestTitForTat(unittest.TestCase):
    """Test case for TitNTat Strategy"""
    def test_copy_cat(self):
        target_player_sims = [Coin.HEAD, Coin.TAIL, Coin.HEAD, Coin.HEAD]
        strat = CopyCat(initial=Coin.HEAD, target_player_sim=target_player_sims)
        perf = strat.strategy()
        self.assertEqual(perf, [Coin.HEAD, Coin.HEAD, Coin.TAIL, Coin.HEAD])

class TestPunisher(unittest.TestCase):
    """Test case for Punisher Strategy"""
    def test_copy_cat(self):
        target_player_sims = [Coin.HEAD, Coin.TAIL, Coin.HEAD, Coin.HEAD]
        strat = Punisher(initial=Coin.HEAD, target_player_sim=target_player_sims)
        perf = strat.strategy()
        self.assertEqual(perf, [Coin.HEAD, Coin.HEAD, Coin.TAIL, Coin.TAIL])
        
class TestUtil(unittest.TestCase):
    """Tests for utilities"""
    def test_get_score(self):
        self.assertEqual(get_score_util(Coin.HEAD, Coin.HEAD), (COOPERATE_WIN_SUM, COOPERATE_WIN_SUM))
        self.assertEqual(get_score_util(Coin.HEAD, Coin.TAIL), (COOPERATE_LOSS_SUM, DEFECT_WIN_SUM))
        self.assertEqual(get_score_util(Coin.TAIL, Coin.HEAD), (DEFECT_WIN_SUM, COOPERATE_LOSS_SUM))
        self.assertEqual(get_score_util(Coin.TAIL, Coin.TAIL), (DEFECT_LOSS_SUM, DEFECT_LOSS_SUM)) # Both rat
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 1.049s

OK


In [165]:
import numpy as np

In [166]:
def get_result(Strat, count, init=Coin.HEAD):
    target_sims = [x for x in TargetPlayer(count)]
    scores = [get_score_util(x,y) for x,y in zip(Strat(init, target_sims).strategy(), target_sims)]
    score_list = list(zip(*scores))
    me, him = [sum(x) for x in score_list]
    return (me, him)

In [167]:
RANGE = 1000
GAME_ROUNDS = 500

In [174]:
# Repeat and Average 500 times
res_tit_for_tat = []
for x in range(RANGE):
    res_tit_for_tat.append(get_result(TitForTat, GAME_ROUNDS))
lst = list(zip(*res_tit_for_tat))
print('me', 'computer')
print(np.mean(lst[0]), np.mean(lst[1]))

me computer
-119.9 244.6


# Analysis - Against an opponent who is random both will be making loss

In [175]:
# Repeat and Average 500 times
res_punisher = []
for x in range(1000):
    res_punisher.append(get_result(Punisher, 500))
lst = list(zip(*res_punisher))
print('me', 'computer')
print(np.mean(lst[0]), np.mean(lst[1]))

me computer
86822.05 -99296.45


# Analysis - Try to squash a random opponent